In [ ]:
# https://blog.csdn.net/atuo200/article/details/119417315?app_version=6.2.8&code=app_1562916241&csdn_share_tail=%7B%22type%22%3A%22blog%22%2C%22rType%22%3A%22article%22%2C%22rId%22%3A%22119417315%22%2C%22source%22%3A%222301_80068426%22%7D&uLinkId=usr1mkqgl919blen&utm_source=app

from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
import torchvision
import numpy as np
import cv2

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

def get_prediction(img_path, threshold):
	#?1 img = Image.open(img_path)
	img = Image.open(img_path).convert('RGB')	#?? RGB image
	transform = T.Compose([T.ToTensor()]) 
	img = transform(img) 
	pred = model([img]) 

	pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())] 
	pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
	pred_score = list(pred[0]['scores'].detach().numpy())
	pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] 
	pred_boxes = pred_boxes[:pred_t+1]
	pred_class = pred_class[:pred_t+1]
	print("pred_class:",pred_class)
	print("pred_boxes:",pred_boxes)
	return pred_boxes, pred_class

def object_detection_api(img_path, threshold=0.5, rect_th=3, text_size=3, text_th=3):
	boxes, pred_cls = get_prediction(img_path, threshold) 
	img = cv2.imread(img_path)
	img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

	for i in range(len(boxes)):
		#4
		cv2.rectangle(img, (int(boxes[i][0][0]), int(boxes[i][0][1])), (int(boxes[i][1][0]), int(boxes[i][1][1])), color=(0, 255, 0),thickness=rect_th)
		cv2.putText(img, pred_cls[i], (int(boxes[i][0][0]), int(boxes[i][0][1])), cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 255, 0), thickness=text_th)
    
		#?2 cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th)
		#2
		# pt1 = tuple(map(int, boxes[i][0]))
		# pt2 = tuple(map(int, boxes[i][1]))
		# cv2.rectangle(img, pt1, pt2, color=(0, 255, 0), thickness=rect_th)

		#?3 cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
		#3
		# org = tuple(map(int, boxes[i][0]))
		# cv2.putText(img, pred_cls[i], org, cv2.FONT_HERSHEY_SIMPLEX, text_size, (0, 255, 0), thickness=text_th)

	plt.imshow(img)
	plt.show()

if __name__ == '__main__':
	object_detection_api(img_path="/Users/chenxin/Desktop/1.png")
	object_detection_api(img_path="/Users/chenxin/Desktop/2.png")
	object_detection_api(img_path="/Users/chenxin/Desktop/3.png")
	object_detection_api(img_path="/Users/chenxin/Desktop/4.png")
	object_detection_api(img_path="/Users/chenxin/Desktop/5.png")
